In [1]:
import pandas as pd
import numpy as np
import re
import os
from os import listdir
from os.path import isfile, join
import data_helpers
from hanziconv import HanziConv

In [54]:
weibo_m_o = pd.read_csv('./data/Weibo_M.csv')
weibo_o = pd.read_csv('./data/Weibo.csv')
wechat_o = pd.read_csv('./data/Wechat.csv')
news_forum_o = pd.read_csv('./data/News_forum.csv')

# Sentiment data

In [55]:
weibo_m_s = weibo_m_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
weibo_s = weibo_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
wechat_s = wechat_o[['CONTENT','SENTIMENT']].dropna(axis=0, how = 'any')

In [56]:
weibo_m_s.columns = ['CONTENT','SENTIMENT']
weibo_s.columns = ['CONTENT','SENTIMENT']

In [57]:
data = pd.concat([weibo_m_s,weibo_s,wechat_s])

In [58]:
data['CONTENT'] = data['CONTENT'].str.replace('\n', '')
data = data.reset_index(drop=True)

In [59]:
length = []
for i in range(len(data)):
    data['CONTENT'][i] = HanziConv.toSimplified(data['CONTENT'][i]).encode('utf-8')
    length.append(len(data['CONTENT'][i]))
    if data['SENTIMENT'][i] == 'Irrelevant':
        data['SENTIMENT'][i] = 'Neutral'
length = np.array(length)

In [60]:
mypath = './data/processed/testing/'
if not os.path.exists(mypath):
    os.makedirs(mypath)
for the_file in os.listdir(mypath):
    file_path = os.path.join(mypath, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [61]:
for x in data['SENTIMENT'].unique():
    np.savetxt(mypath+ x.replace('/',' ') +'.utf8',\
                   data[data['SENTIMENT'] == x]['CONTENT'].values, fmt='%s')

In [63]:
mypath = './data/processed/training/'
if not os.path.exists(mypath):
    os.makedirs(mypath)
for the_file in os.listdir(mypath):
    file_path = os.path.join(mypath, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [62]:
data['SENTIMENT'].value_counts()

Neutral     14871
Positive      465
Negative      287
Name: SENTIMENT, dtype: int64

In [1]:
from gensim.models import Word2Vec
w2vModel = Word2Vec.load('runs/trained_word2vec.model')

In [2]:
len(w2vModel.wv.vocab)

3577

In [68]:
from gensim.models import KeyedVectors
w2vModel = KeyedVectors.load_word2vec_format('runs/wiki.zh.vec')

In [53]:
vocab_obj = w2vModel.wv.vocab
word = u'河北'
print word
print vocab_obj[word].index

河北
3038


In [2]:
dianping_json = pd.read_table('./data/reviews.txt')

In [ ]:
content = []
rate = []


for i in range(len(dianping_json)):
    try:
        x = (pd.read_json(dianping_json.loc[i][0].split('^')[1], lines=True))
        content.append(x['content'][0])
        rate.append(x['rate'][0])
    except:
        pass

In [ ]:
dianping = pd.DataFrame([content,rate],columns=['content','rate'])

In [ ]:
dianping